# Speeding Up the Search for "Compromise Solutions" Before It's Too Late
### A walk through on Surrogate-Assisted MOO (SAMOO)

Now that we know Pareto optimality and that there's a tool to efficiently perform MOO, we have to face another problem: such undertaking is computationally expensive -- for large-scale real-world numerical model, one such forward run could take hours, let alone hundreds of thousand of runs required in MOO.

For this tutorial, we will demonstrate how we can speed up the process of MOO by using a surrogate model, without compromising the reliability of solutions. We will still be using the Fonseca-Fleming problem.

In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyemu
import glob


Recall that the Fonseca-Fleming problem has two objectives with different decision variable values for their respective minimums

In [2]:
def fonseca_fleming(x):
    x = np.array(x)
    n = len(x)
    # Fonseca-Fleming objective functions
    # f1(x) = 1 - exp(-sum((xi - 1/sqrt(n))^2))
    # f2(x) = 1 - exp(-sum((xi + 1/sqrt(n))^2))
    term1 = np.sum((x - 1/np.sqrt(n))**2)
    term2 = np.sum((x + 1/np.sqrt(n))**2)
    
    obj1 = 1 - np.exp(-term1)
    obj2 = 1 - np.exp(-term2)
    
    return obj1, obj2

n_samples = 1000
x1_range = np.linspace(-4, 4, int(np.sqrt(n_samples)))
x2_range = np.linspace(-4, 4, int(np.sqrt(n_samples)))
x1, x2 = np.meshgrid(x1_range, x2_range)
x1 = x1.flatten()
x2 = x2.flatten()

x1 = x1[:n_samples]
x2 = x2[:n_samples]

n_actual = min(len(x1), len(x2))
objectives = np.array([fonseca_fleming([x1[i], x2[i]]) for i in range(n_actual)])
obj1 = objectives[:, 0]
obj2 = objectives[:, 1]

Let us first generate the template files we need.

In [ ]:
base_d = "./base_files"
assert os.path.exists(base_d)

temp_d = "fonseca_fleming_demo"
if os.path.exists(temp_d):
    shutil.rmtree(temp_d)
shutil.copytree(base_d,temp_d)

We can easily find Pareto optimal set of decision variables by running PESTPP-MOU. Let's generate an initial swarm size of 30 and perform 20 iterations of MOO. To ensure that our initial population is evenly distributed and sufficiently samples the decision space, it is advisable to use Latin Hypercube Sampling (LHS).

In [ ]:
# You can change the size of initial sample or the number of inner iterations if you want to try other values. 
# Just be aware that larger sample size and more inner iterations will take longer time to run.
swarm_size = 30
nmax_inner = 20

sys.path.append(temp_d)
from LHS_sampler import generate_lhsstarter
bounds = np.array([[-4, 4] for i in range(2)])
generate_lhsstarter(os.path.join(temp_d,'fon_pbm_template'), seed=42, n_samples=swarm_size, n_dimensions=2, bounds=bounds) 

pst_file = os.path.join(temp_d, 'fon_pbm_template', 'fon.pst')
pst = pyemu.Pst(pst_file)
pst.pestpp_options['mou_population_size'] = swarm_size
pst.pestpp_options['mou_dv_population_file'] = 'starter.dv_pop.csv'
pst.control_data.noptmax = nmax_inner
pst.write(os.path.join(temp_d, 'fon_pbm_template', os.path.basename(pst_file)))

starter_pop = pd.read_csv(os.path.join(temp_d, "fon_pbm_template", "starter.dv_pop.csv"), index_col="real_name")
starter_pop.head()

Now that we have generated an initial population, let us run pestpp-mou...

In [5]:
num_workers = 8
tmpl_in = os.path.join(temp_d, "fon_pbm_template")
sys.path.insert(0,tmpl_in)
from forward_pbrun import ppw_worker as ppw_function
pyemu.os_utils.start_workers(tmpl_in, "pestpp-mou", "fon.pst", num_workers = num_workers,
                             worker_root = temp_d, master_dir = os.path.join(temp_d, "pbm_run"),
                             ppw_function = ppw_function)
sys.path.remove(tmpl_in)

Let's plot the resulting Pareto front. Use the slider to observe how the swarm's position evolves in the objective space at each iteration as it searches for the Pareto front until it eventually converge. Also observe how the swarm moves in the decision space.

We will revisit this later to compare with SAMOO, which will be introduced next. This will be referred to as Complex MOO run.

In [ ]:
from ipywidgets import interact, IntSlider, HBox, VBox, Output
from IPython.display import display
from scipy.interpolate import griddata
import numpy as np
import glob
import matplotlib.pyplot as plt

run_data = pd.read_csv(os.path.join(temp_d, "pbm_run", "fon.pareto.summary.csv"))
max_gen = max(run_data['generation'])

csvfiles = sorted(glob.glob(os.path.join(temp_d, "pbm_run", "*[0-999].dv_pop.csv"), recursive=True), 
                  key=lambda x: int(x.split(".dv")[0].split(".")[1]))
all_dv_list = []
for file in csvfiles:
    generation = int(file.split(".dv")[0].split(".")[1])
    df = pd.read_csv(file).assign(generation=generation)
    df = df[['generation'] + [col for col in df.columns if col != 'generation']] 
    all_dv_list.append(df)
all_dv = pd.concat(all_dv_list, ignore_index=True)

csvfiles = sorted(glob.glob(os.path.join(temp_d, "pbm_run", "*[0-999].obs_pop.csv"), recursive=True), 
                      key=lambda x: int(x.split(".obs")[0].split(".")[1]))

all_obs_list = []
for file in csvfiles:
    generation = int(file.split(".obs")[0].split(".")[1])
    df = pd.read_csv(file).assign(generation=generation)
    df = df[['generation'] + [col for col in df.columns if col != 'generation']] 
    all_obs_list.append(df)
all_obs = pd.concat(all_obs_list, ignore_index=True)

all_data = pd.merge(all_dv, all_obs, on=['generation', 'real_name'])
max_gen = max(all_data['generation'])

x = all_data['x1'].values
y = all_data['x2'].values
z1 = all_data['obj1'].values
z2 = all_data['obj2'].values

xi = np.linspace(min(x), max(x), 100)
yi = np.linspace(min(y), max(y), 100)
xi_grid, yi_grid = np.meshgrid(xi, yi)

zi1 = griddata((x, y), z1, (xi_grid, yi_grid), method='cubic')
zi2 = griddata((x, y), z2, (xi_grid, yi_grid), method='cubic')

out_pareto = Output()
out_obj_space = Output()
out_contour = Output()

true_pareto = pd.read_csv(os.path.join('fonseca_fleming_demo', 'fonseca_fleming_solution_obj.csv'))

def moo_plot_all(generation):

    with out_pareto:
        out_pareto.clear_output(wait=True)
        pareto = run_data.loc[(run_data['generation']==generation) & (run_data['nsga2_front'] == 1)]
        plt.figure(figsize=(6, 4))
        plt.scatter(pareto['obj1'], pareto['obj2'], c='firebrick', s=50, alpha=0.7, label = 'current non-dominated positions')
        plt.scatter(true_pareto['obj1'], true_pareto['obj2'], c='deepskyblue', s=20, zorder = -10, label='Pareto optimal solutions')
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.legend()
        plt.title(f'Pareto Front at Generation {generation}')
        plt.tight_layout()
        plt.show()

    with out_obj_space:
        out_obj_space.clear_output(wait=True)
        all_points = run_data.loc[(run_data['generation']==generation)]
        plt.figure(figsize=(6, 4))
        plt.scatter(all_points['obj1'], all_points['obj2'], edgecolor='green', c = 'none', s=50, alpha=0.7, label = 'all candidate positions')
        plt.scatter(true_pareto['obj1'], true_pareto['obj2'], c='deepskyblue', s=20, zorder = -10, label='Pareto optimal solutions')
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.legend()
        plt.title(f'Objective Space at Generation {generation}')
        plt.tight_layout()
        plt.show()

    with out_contour:
        out_contour.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(8, 6))
        contour1 = ax.contourf(xi_grid, yi_grid, zi1, 15, cmap='plasma', alpha=0.5)
        fig.colorbar(contour1, ax=ax, label='Objective 1')
        contour2 = ax.contour(xi_grid, yi_grid, zi2, 15, cmap='viridis', alpha=0.7)
        fig.colorbar(contour2, ax=ax, label='Objective 2')

        pareto_members = pareto['member'].values
        pareto_dv = all_data[all_data['real_name'].isin(pareto_members)]
        ax.scatter(pareto_dv['x1'], pareto_dv['x2'], edgecolor='firebrick', facecolor='none', s=40, alpha=0.8)
        
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_title(f'Pareto Optimal Decisions at Generation {generation}')
        ax.grid(True)
        plt.tight_layout()
        plt.show()

generation_slider = IntSlider(min=0, max=max_gen, step=1, value=max_gen, 
                             description='Generation:',
                             continuous_update=False)
moo_plot_all(max_gen)

generation_slider.observe(lambda change: moo_plot_all(change['new']), names='value')
display(VBox([
    generation_slider,
    HBox([out_obj_space, out_pareto]),
    out_contour
]))


As this benchmark is really easy, it doesn't take much time to finish the entire optimization process and it doesn't take a lot of iteration to obtain a set of Pareto optimal solutions. However, in reality, real-world models do not run this fast. Even with parallelisation, it can take a few hundred, even thousand iterations until the swarm converges to the Pareto front. If important timely decisions rely on the outcome of such optimisation process, we would need some help to speed up this process. 

Let's pretend for a while that the Fonseca-Fleming problem is a complex model that is expensive to evaluate. There's nothing we could do about the run time of the complex model. What we can do though is to employ an "emulator", a faster and cheaper model that approximates the relationships between the known (i.e., previously evaluated) input and outputs of the complex model in order to predict its response to some input values that were not previously evaluated. This emulator is also known as the surrogate model. Because the surrogate model is an approximation, its prediction has errors. These errors could mislead the decision-making to some suboptimal solutions that are not actually in the Pareto front. This is some price to pay for speeding up the process, but don't worry as we have means to manage these uncertainties and still obtain truly Pareto optimal solutions.

There are many surrogate models available in literature. However, we will use Gaussian Process Regression (GPR) as it provides a convenient way to take into account the uncertainty in predictions of the surrogate model, which, as previously said, needs to be managed well.

Surrogate-Assisted Multi-Objective Optimisation (SAMOO) follows this general algorithm:
1. LHS sampling of initial training dataset and starting population
2. Complex model evaluation (hereinafter referred to as Outer Iteration)
3. Pareto dominance evaluation
     - if front converged: exit; else: continue to step 4
4. (Re)training the GPR
5. MOO with GPR replacing the complex model (hereinafter referred to as Inner Iterations)
6. Resampling for new training points (also called infills)

Steps 2-6 are performed iteratively until convergence is achieved in Step 3. 

To facilitate this entire process, we will use samoo_utils.py. Let us first prepare our PEST files (control, template, instruction files) to be SAMOO-ready. Let's create a new directory where we will put all our SAMOO-related files and generate an initial training dataset for the surrogate model using Latin hypercube sampling.


In [7]:
if os.path.exists(os.path.join(temp_d, 'samoo_runs')):
    shutil.rmtree(os.path.join(temp_d, 'samoo_runs'))
os.makedirs(os.path.join(temp_d, 'samoo_runs'))
shutil.copytree(os.path.join(temp_d, 'fon_samoo_template'), os.path.join(temp_d, 'samoo_runs', 'template'))

sys.path.insert(0,temp_d)
from samoo_utils import SAMOO
optimizer = SAMOO() #load the utility and its default values for now. We modify some args below
optimizer.nmax_inner = 20 #the number of inner iterations
optimizer.ppd_beta = 0.7 #this is explained later


from LHS_sampler import generate_lhstrainingset
generate_lhstrainingset(os.path.join(temp_d, 'samoo_runs', 'template'), seed=42, n_samples=swarm_size, n_dimensions=2, bounds=bounds)

sys.path.remove(temp_d)

Let's prepare all template files needed. We will generate three sets of template files:
1. template_outer -- for the outer iteration
2. template_inner -- for the inner iteration
3. template_repo_update -- for updating the outer iteration repository

In [ ]:
os.chdir(os.path.join('fonseca_fleming_demo', 'samoo_runs'))
optimizer.prep_templates()

Then, run a sweep of the initial training dataset through the complex model to get the deterministic output with which to train the GPR with.  Result of this will be saved in the directory "Outer_0". This is Step 2-3 of the algorithm.

In [ ]:
optimizer.outer_sweep(0)

In [ ]:
def plot_outer(outer_dir, fig=None, ax=None, show=True):
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if outer_dir.endswith('_0'):
        outer_pareto = pd.read_csv(os.path.join(outer_dir, 'fon.pareto.summary.csv'))
    else:
        outer_pareto = pd.read_csv(os.path.join(outer_dir, 'outer_repo', 'outer_repo.pareto.summary.csv'))
    
    outer_pareto = outer_pareto.loc[outer_pareto['nsga2_front'] == 1]
    
    ax.scatter(outer_pareto['obj1'], outer_pareto['obj2'], c='none', ec='firebrick', s=20, zorder=10, label='Outer repository positions')
    ax.scatter(true_pareto['obj1'], true_pareto['obj2'], c='deepskyblue', s=20, label='Pareto optimal solutions')
    
    ax.set_xlabel('Objective 1')
    ax.set_ylabel('Objective 2')
    ax.legend()
    
    ax.set_title(f'Outer-{outer_dir.split("_")[1]}')
    if show and fig is not None:
        plt.tight_layout()
        plt.show()
    
    return fig, ax, outer_pareto

plot_outer('outer_0')


Outer-0 produced some non-dominated positions. They are still far from the true Pareto optimal solutions.

The input-output pair resulting from such sweep will be used as training dataset to retrain the GPR, which will be needed in the inner iterations. This GPR will be called in the inner iterations instead of the complex model.

Let's prepare the GPR training files (Step 4) and proceed with the first inner iteration (Step 5), the output of which will be saved in "Inner_1" directory. We will do 20 inner iterations for now.

In [ ]:
inner_dirs, outer_dirs = optimizer.get_dirlist()
optimizer.inner_prep(inner_dirs, outer_dirs) #this will write the training dataset
next_inner_index = 1 if len(inner_dirs) == 0 else int(inner_dirs[-1].split("_")[1]) + 1
optimizer.inner_opt(next_inner_index) #this execute pestpp-mou that calls on the GPR in the inner iterations

At this stage the GPR only has very limited knowledge informed only by a sample of 30 points from the decision space and their corresponding deterministic objective function values. Thus, we recognize that GPR emulations are incorrect and these errors should be taken into consideration when evaluating dominance relationships in the inner iterations. SAMOO deals with these uncertainties through "Probabilistic Pareto Dominance" or PPD. The basic idea of PPD is that we cannot rule out dominated members to be completely suboptimal, likewise, we cannot trust that non-dominated members are truly Pareto optimal. Thus, if the dominance relationships does not reach our confidence threshold (passed through the ppd_beta argument in the pestpp-mou control file), we can just keep both solution in the 'surrogate repository' for now. As a result, we won't have a Pareto 'front' in the inner iterations but we will have as fuzzy set of solutions all of which could 'probably' be in the true front. We call this the "Pareto cloud". The cloudiness of the Pareto cloud is determined by the value of ppd_beta. In our experience, a ppd_beta between 0.5-0.7 generally works well in most cases. In this demo, we are using 0.6.

Let's see what the Pareto cloud looks like through the inner iterations.

In [ ]:
inner_dirs, outer_dirs = optimizer.get_dirlist()

def plot_inner(inner_dir, iteration, ax = None, fig = None, show=True):
    if ax is None or fig is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    inner_rundata = pd.read_csv(os.path.join(inner_dir, 'fon.pareto.summary.csv'))
    a = ax.scatter(true_pareto['obj1'], true_pareto['obj2'], c='deepskyblue', s = 20, label = 'Pareto optimal solutions')

    pareto_cloud = inner_rundata.loc[(inner_rundata['generation'] == iteration) & (inner_rundata['nsga2_front'] == 1)]
    b = ax.scatter(pareto_cloud['obj1'], pareto_cloud['obj2'], c='none', ec='g', s=30, label='Pareto cloud')

    all_dv, all_obs = optimizer.parse_all_io(inner_dir)
    pareto_dv = all_dv.loc[all_dv['real_name'].isin(pareto_cloud['member'])]

    from matplotlib.patches import Ellipse
    from matplotlib.patches import Patch
    for mem in pareto_cloud['member']:
        o1 = pareto_cloud.loc[pareto_cloud['member'] == mem, 'obj1'].item()
        o2 = pareto_cloud.loc[pareto_cloud['member'] == mem, 'obj2'].item()
        ellipse_ab = all_obs.loc[all_obs['real_name'] == mem]
        o1sd = ellipse_ab['obj1_sd'].item()
        o2sd = ellipse_ab['obj2_sd'].item()
        ellipse = Ellipse((o1, o2), width=3*o1sd, height=3*o2sd, fc='peachpuff', zorder=-10)
        ax.add_artist(ellipse)
    
    sdpatch = Patch(facecolor='peachpuff', edgecolor='none', label='$\pm 3$ sd')
    ax.legend(handles=[a, b, sdpatch], loc=0)
    ax.set_xlabel('Objective 1')
    ax.set_ylabel('Objective 2')
    ax.set_title(f'Inner-{iteration}')

    if show and fig is not None:
        plt.tight_layout()
        plt.show()

    return fig, ax, pareto_dv, pareto_cloud

import ipywidgets as widgets

def update_plots(iteration):
    plt.close('all')
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot 1: Pareto Cloud with uncertainty ellipse
    _,_,pareto_dv,_ = plot_inner(inner_dirs[-1], iteration, ax = axes[0], fig = fig, show = False)

    # Collect data for surface plots
    training_dv = pd.read_csv(os.path.join(inner_dirs[-1], 'gp_0.dv_training.csv'))
    training_obs = pd.read_csv(os.path.join(inner_dirs[-1], 'gp_0.obs_training.csv'))
    training_data = pd.merge(training_dv, training_obs, on='real_name', how='inner')
    
    x1_train = training_data['x1'].values
    x2_train = training_data['x2'].values
    obj1_train = training_data['obj1'].values
    obj2_train = training_data['obj2'].values
    
    grid_size = 100
    x1_grid = np.linspace(-4, 4, grid_size)
    x2_grid = np.linspace(-4, 4, grid_size)
    X1, X2 = np.meshgrid(x1_grid, x2_grid)
    
    from scipy.interpolate import griddata
    Z1 = griddata((x1_train, x2_train), obj1_train, (X1, X2), method='cubic')
    Z2 = griddata((x1_train, x2_train), obj2_train, (X1, X2), method='cubic')
    
    # Plot 2: Objective 1 Surface
    im1 = axes[1].contourf(X1, X2, Z1, cmap='plasma', levels=20)
    plt.colorbar(im1, ax=axes[1], label='Objective 1')
    axes[1].scatter(x1_train, x2_train, c='grey', ec='none', s=20, alpha=0.5, label='training points')
    axes[1].scatter(pareto_dv['x1'], pareto_dv['x2'], c='none', ec='g', s=30, label='dv of Pareto cloud')
    axes[1].set_xlabel('x1')
    axes[1].set_ylabel('x2')
    axes[1].set_title('Objective 1 Surface in Decision Space')
    axes[1].legend()
    
    # Plot 3: Objective 2 Surface
    im2 = axes[2].contourf(X1, X2, Z2, cmap='plasma', levels=20)
    plt.colorbar(im2, ax=axes[2], label='Objective 2')
    axes[2].scatter(x1_train, x2_train, c='grey', ec='none', s=20, alpha=0.5, label='training points')
    axes[2].scatter(pareto_dv['x1'], pareto_dv['x2'], c='none', ec='g', s=30, label='dv of Pareto cloud')
    axes[2].set_xlabel('x1')
    axes[2].set_ylabel('x2')
    axes[2].set_title('Objective 2 Surface in Decision Space')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

slider = widgets.IntSlider(
    value=optimizer.nmax_inner,
    min=0,
    max=optimizer.nmax_inner,
    step=1,
    description='Inner Iteration:',
    continuous_update=False
)
widgets.interactive(update_plots, iteration=slider)


Notice how the confidence ellipsoids of the Pareto cloud positions overlap significantly. If we simply follow the conventional Pareto dominance sorting, many of these positions would be dominated and discarded even if they may potentially be non-dominated considering their uncertainty. How many of these do you think will dominate those that are already in the repository from Outer-0?

Also observe that the Pareto cloud cannot get close any further to the true Pareto front. This is a result of the limited knowledge of the GPR at this stage. We must therefore retrain the GPR to improve its prediction. The Pareto cloud contains the most informative solutions that, once evaluated through the complex model, are either nearly or already truly Pareto optimal solutions. We can therefore sample the Pareto cloud for decision variable values that are worth evaluating in the expensive complex model (also known as the infill points), which will then be added to the existing knowledge base (i.e., the training dataset) of the GPR and retrain the GPR with this updated information.

Let's proceed with sampling this Pareto cloud for 30 infills -- this is Step 6 of the algorithm.

In [ ]:
inner_dirs, outer_dirs = optimizer.get_dirlist()
optimizer.max_infill = swarm_size
optimizer.resample(inner_dirs, outer_dirs)

We go back to Step 2-3 -- evaluate these infills through the complex model, which will be the Outer-1, and perform dominance evaluation of Pareto optimal solutions obtained in Outer-0 and Outer-1.

In [14]:
outer_dirs = optimizer.outer_sweep(1)
optimizer.update_outer_repo(outer_dirs)

Did we make some progress in finding better solutions than those currently in the repository from Outer-0?

In [ ]:
inner_dirs, outer_dirs = optimizer.get_dirlist()

def plot_comparison(outer_selection):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    if outer_selection == 'outer_0':
        plot_outer('outer_0', ax=axes[1], fig=fig, show=False)
        axes[0].set_title("No inner plot")
        axes[0].set_xlabel("Objective 1")
        axes[0].set_ylabel("Objective 2")
    else:
        outer_num = outer_selection.split('_')[1]
        matching_inner_dir = next((inner_dir for inner_dir in inner_dirs if inner_dir.endswith(f"_{outer_num}")), inner_dirs[-1])
        _,_,_,pareto_cloud = plot_inner(matching_inner_dir, optimizer.nmax_inner, ax=axes[0], fig=fig, show=False)
        _,_,outer_pareto = plot_outer(outer_selection, ax=axes[1], fig=fig, show=False)
        
        successful_infill = pareto_cloud.loc[pareto_cloud['member'].isin(outer_pareto['member'])]
        axes[0].scatter(successful_infill['obj1'], successful_infill['obj2'], 
                       c='firebrick', ec='none', s=30, label='successful infill')
        axes[0].legend()
    
    plt.tight_layout()
    return fig

outer_dropdown = widgets.Dropdown(
    options=outer_dirs,
    value='outer_1',
    description='Outer Iteration:',
    disabled=False
)

# Display the interactive plot
widgets.interactive(plot_comparison, outer_selection=outer_dropdown)

That's definitely a lot of progress there as many of the infills selected are actually sitting at the true front even though the GPR prediction was incorrect. Also notice that some of the successful infill would have been discarded if we used the conventional Pareto dominance sorting and treat the GPR predicted positions as deterministic. We would have missed out really informative positions.

Let's do 3 more outer iterations

In [ ]:
optimizer.restart = True #we will pickup from where we left off at Outer-1
optimizer.nmax_outer = 3 #3 more outer iterations
optimizer.run() #run the whole SAMOO process outlined above

Let's look at how we progressed in each outer iterations. Compare the resulting outer repository positions at each outer iteration. Notice how the confidence ellipsoid is reduced significantly in the inner iterations as more infills are added in the training dataset of the GPR.

In [ ]:
inner_dirs, outer_dirs = optimizer.get_dirlist()

outer_dropdown = widgets.Dropdown(
    options=outer_dirs,
    value=outer_dirs[-1],
    description='Outer Iteration:',
    disabled=False
)

# Display the interactive plot
widgets.interactive(plot_comparison, outer_selection=outer_dropdown)

Now, let's revisit the progress of the Complex MOO run that we did earlier. Since GPR emulations are way faster compared to the complex model, we can "roughly" compare the progress of the Complex MOO run and SAMOO by their repository positions after having run the same number of complex model evaluations -- that is, the regular iteration of complex MOO vs the outer iteration of MOO.

Use the slider below to plot Iteration-1 of complex MOO and compare it with Outer-1 of SAMOO in the plots above. Do you see the difference? SAMOO is able to find objective positions that are near or already Pareto optimal with fewer complex model evaluations. Keep comparing other iterations. 

In [ ]:
generation_slider = IntSlider(min=0, max=max_gen, step=1, value=max_gen, 
                             description='Generation:',
                             continuous_update=False)
moo_plot_all(max_gen)

generation_slider.observe(lambda change: moo_plot_all(change['new']), names='value')
display(VBox([
    generation_slider,
    HBox([out_obj_space, out_pareto]),
    out_contour
]))

How many complex MOO iterations does it take to make it at the same level of progress that SAMOO had in Outer-1? Remember we are just pretending that the Fonseca-Fleming problem is numerically expensive and complex, so run times for both these algorithms don't differ much, but imagine how much computational budget you would have saved if the Fonseca-Fleming functions were real-world groundwater model with each forward run taking hours to complete.

### References

Macasieb, R. Q., White, J. T., Pasetto, D., & Siade, A. J. (2025). A probabilistic approach to surrogate‐assisted multi‐objective optimization of complex groundwater problems (in production). Water Resources Research, 61, e2024WR038554. https://doi.org/10.1029/2024WR038554